In [1]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())


                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [17]:
import pandas as pd

# CSV 또는 Parquet 파일을 불러온 뒤 df에 저장했다고 가정
# 예: df = pd.read_csv('your_file.csv') 또는 pd.read_parquet('your_file.parquet')

#  1. price가 0인 행 제거
df = df[df['price'] > 0]

#  2. brand 컬럼에서 결측치 제거
df = df.dropna(subset=['brand'])

# 결과 확인 (선택사항)
print(df.shape)
print(df['price'].min())
print(df['brand'].isnull().sum())


(11932733, 12)
0.06
0


In [16]:
# 1. 고객별 첫 등장 월 계산
df['first_month'] = df.groupby('user_key')['event_month'].transform('min')

# 2. 신규 고객: 등장한 월 == 첫 등장 월
new_customer_by_month = df[df['event_month'] == df['first_month']]

# 3. 월별 신규 고객 수 계산
monthly_new = new_customer_by_month.groupby('event_month')['user_key'].nunique().reset_index()
monthly_new.columns = ['event_month', 'new_customer_count']

# 4. CSV로 저장
monthly_new.to_csv('monthly_new_customers.csv', index=False)

# 5. 결과 출력
print("✅ 'monthly_new_customers.csv' 파일로 저장 완료!")
print("\n📊 월별 신규 고객 수:")
print(monthly_new)



✅ 'monthly_new_customers.csv' 파일로 저장 완료!

📊 월별 신규 고객 수:
   event_month  new_customer_count
0          1.0              659031
1          2.0              632050
2         10.0              588996
3         11.0              622935
4         12.0              562466


In [15]:
# 1. 고객별로 방문한 월 목록 추출
user_months = df.groupby('user_key')['event_month'].unique()

# 2. 각 고객이 재방문자인지 여부 (2개월 이상 방문한 고객)
repeat_users = user_months[user_months.apply(lambda x: len(x) >= 2)].index

# 3. 각 월별 전체 고객 수와 재방문 고객 수 계산
monthly_user = df.groupby('event_month')['user_key'].nunique()
monthly_repeat = df[df['user_key'].isin(repeat_users)].groupby('event_month')['user_key'].nunique()

# 4. 비율 계산
monthly_repeat_ratio = (monthly_repeat / monthly_user * 100).reset_index()
monthly_repeat_ratio.columns = ['event_month', 'repeat_customer_ratio']

# 5. 소수점 2자리로 정리
monthly_repeat_ratio['repeat_customer_ratio'] = monthly_repeat_ratio['repeat_customer_ratio'].round(2)

# 6. CSV 저장
monthly_repeat_ratio.to_csv('monthly_repeat_customer_ratio.csv', index=False)

# 7. 결과 출력
print("✅ 'monthly_repeat_customer_ratio.csv' 파일로 저장 완료!")
print("\n📊 월별 재방문 고객 비율:")
print(monthly_repeat_ratio)



✅ 'monthly_repeat_customer_ratio.csv' 파일로 저장 완료!

📊 월별 재방문 고객 비율:
   event_month  repeat_customer_ratio
0          1.0                   0.73
1          2.0                   1.10
2         10.0                   0.43
3         11.0                   0.64
4         12.0                   0.58


In [14]:
# 1. 월별 고객 목록 딕셔너리 생성
month_customer = df.groupby('event_month')['user_key'].unique().to_dict()

# 2. 월별 고객 이탈률 계산
churn_data = []

months = sorted(month_customer.keys())
for i in range(1, len(months)):
    prev_month = months[i - 1]
    curr_month = months[i]
    
    prev_users = set(month_customer[prev_month])
    curr_users = set(month_customer[curr_month])
    
    # 이탈한 고객 = 이전 달에는 있었지만 이번 달에는 없는 고객
    churned_users = prev_users - curr_users
    churn_rate = len(churned_users) / len(prev_users) * 100 if len(prev_users) > 0 else 0

    churn_data.append({
        'event_month': curr_month,
        'churn_rate': round(churn_rate, 2)
    })

# 3. DataFrame으로 변환
churn_df = pd.DataFrame(churn_data)

# 4. CSV 저장
churn_df.to_csv('monthly_churn_rate.csv', index=False)

# 5. 결과 출력
print("✅ 'monthly_churn_rate.csv' 파일로 저장 완료!")
print("\n📊 월별 고객 이탈률:")
print(churn_df)



✅ 'monthly_churn_rate.csv' 파일로 저장 완료!

📊 월별 고객 이탈률:
   event_month  churn_rate
0          2.0       99.42
1         10.0       99.81
2         11.0       99.74
3         12.0       99.75


In [13]:
# 1. 구매 이벤트만 필터링 (event_type == 'purchase')
purchase_df = df[df['event_type'] == 'purchase']

# 2. 고객별 총 구매 금액 (CLV)
clv_df = purchase_df.groupby('user_key')['price'].sum().reset_index()
clv_df.columns = ['user_key', 'CLV']

# 3. 소수점 반올림
clv_df['CLV'] = clv_df['CLV'].round(2)

# 4. CSV 저장
clv_df.to_csv('customer_lifetime_value.csv', index=False)

# 결과 출력
print("✅ 'customer_lifetime_value.csv' 파일로 저장 완료!")
print("\n📊 고객별 생애가치(CLTV) 상위 10명:")
print(clv_df.sort_values(by='CLV', ascending=False).head(10))


✅ 'customer_lifetime_value.csv' 파일로 저장 완료!

📊 고객별 생애가치(CLTV) 상위 10명:
                                             user_key      CLV
14465   396059467285e8547-29b3-49d2-b503-5ca9a60413cc  1080.97
76512   5578507438db8915d-ab78-4147-8c32-cde983be715d   957.14
63116   54351257616e3af43-c113-4a04-afa5-83f4f9f735b8   929.57
97062   572631412a133acd7-8cf6-e069-e38d-303c4104c3da   874.52
16852   409977933d09dd883-2ee5-4031-b8b3-8b5ceee72d05   872.18
29879   463845330d9c660ad-313f-401b-bf2e-53c1e8b77b11   849.32
65431   546827800346dd83e-e8f0-69df-085a-0cf4a806d70d   827.12
39535   491009486028266ad-9d7a-4ad9-9aac-93884e5a875e   800.64
131067  6103270418a70ec89-41f3-4733-9e74-2710ce5380fb   790.34
48808   5182780600a18130a-9230-4d34-a78c-b1bf13d928d6   751.62


In [12]:
# 월별 사용자 수 및 세션 수 집계
monthly_counts = df.groupby('event_month').agg({
    'user_key': pd.Series.nunique,
    'user_session': pd.Series.nunique
}).reset_index()

# 컬럼명 정리
monthly_counts.columns = ['event_month', 'unique_users', 'unique_sessions']

# CSV 저장
monthly_counts.to_csv('monthly_user_session_counts.csv', index=False)

# 출력
print("✅ 'monthly_user_session_counts.csv' 파일로 저장 완료!")
print("\n📊 월별 사용자 및 세션 수 요약:")
print(monthly_counts)



✅ 'monthly_user_session_counts.csv' 파일로 저장 완료!

📊 월별 사용자 및 세션 수 요약:
   event_month  unique_users  unique_sessions
0          1.0        659031           658956
1          2.0        635873           635804
2         10.0        590517           590495
3         11.0        626152           626128
4         12.0        565773           565745


In [18]:
# View → Cart
view_users = set(df[df['event_type'] == 'view']['user_key'])
cart_users = set(df[df['event_type'] == 'cart']['user_key'])
drop_view_to_cart = view_users - cart_users
view_to_cart_drop_rate = len(drop_view_to_cart) / len(view_users) * 100

# Cart → Purchase
purchase_users = set(df[df['event_type'] == 'purchase']['user_key'])
drop_cart_to_purchase = cart_users - purchase_users
cart_to_purchase_drop_rate = len(drop_cart_to_purchase) / len(cart_users) * 100

# 출력
print(f"📉 View→Cart 이탈률: {view_to_cart_drop_rate:.2f}%")
print(f"📉 Cart→Purchase 이탈률: {cart_to_purchase_drop_rate:.2f}%")

# 가장 큰 구간 강조
if view_to_cart_drop_rate > cart_to_purchase_drop_rate:
    max_drop = "View → Cart"
else:
    max_drop = "Cart → Purchase"

print(f"✅ 가장 큰 이탈 구간: {max_drop}")

# 결과 보기: DataFrame으로 정리
import pandas as pd

drop_summary = pd.DataFrame([{
    'view_to_cart_drop_rate (%)': round(view_to_cart_drop_rate, 2),
    'cart_to_purchase_drop_rate (%)': round(cart_to_purchase_drop_rate, 2),
    'max_drop_stage': max_drop
}])

# CSV 저장
drop_summary.to_csv('overall_dropoff_summary.csv', index=False)

# 결과 출력
print("\n📊 전체 이탈률 요약:")
print(drop_summary)
print("✅ 'overall_dropoff_summary.csv' 파일로 저장 완료!")




📉 View→Cart 이탈률: 80.98%
📉 Cart→Purchase 이탈률: 85.81%
✅ 가장 큰 이탈 구간: Cart → Purchase

📊 전체 이탈률 요약:
   view_to_cart_drop_rate (%)  cart_to_purchase_drop_rate (%)   max_drop_stage
0                       80.98                           85.81  Cart → Purchase
✅ 'overall_dropoff_summary.csv' 파일로 저장 완료!


In [10]:
# 결과 저장 리스트
monthly_dropoff = []

# 분석할 월 목록
months = sorted(df['event_month'].dropna().unique())

for month in months:
    month_df = df[df['event_month'] == month]
    
    # 월별 유저 집합
    view_users = set(month_df[month_df['event_type'] == 'view']['user_key'])
    cart_users = set(month_df[month_df['event_type'] == 'cart']['user_key'])
    purchase_users = set(month_df[month_df['event_type'] == 'purchase']['user_key'])
    
    # View → Cart 이탈
    drop_view_to_cart = view_users - cart_users
    view_to_cart_drop_rate = len(drop_view_to_cart) / len(view_users) * 100 if len(view_users) > 0 else 0

    # Cart → Purchase 이탈
    drop_cart_to_purchase = cart_users - purchase_users
    cart_to_purchase_drop_rate = len(drop_cart_to_purchase) / len(cart_users) * 100 if len(cart_users) > 0 else 0

    # 저장
    monthly_dropoff.append({
        'event_month': int(month),
        'view_to_cart_drop_rate': round(view_to_cart_drop_rate, 2),
        'cart_to_purchase_drop_rate': round(cart_to_purchase_drop_rate, 2)
    })

# DataFrame 변환
dropoff_df = pd.DataFrame(monthly_dropoff)

# CSV 저장
dropoff_df.to_csv('monthly_dropoff_rates.csv', index=False)

# 결과 출력
print("✅ 'monthly_dropoff_rates.csv' 파일로 저장 완료!")
print("\n📊 월별 이탈률 데이터:")
print(dropoff_df)



✅ 'monthly_dropoff_rates.csv' 파일로 저장 완료!

📊 월별 이탈률 데이터:
   event_month  view_to_cart_drop_rate  cart_to_purchase_drop_rate
0            1                   82.30                       85.29
1            2                   83.03                       86.25
2           10                   76.43                       87.91
3           11                   79.84                       84.79
4           12                   83.04                       84.41


In [19]:
# 유저별 이벤트 카운트
user_event_counts = df.pivot_table(index='user_key', 
                                   columns='event_type', 
                                   aggfunc='size', 
                                   fill_value=0).reset_index()

# 전환율 계산: purchase / view
user_event_counts['conversion_rate'] = (user_event_counts['purchase'] / user_event_counts['view']) * 100
user_event_counts['conversion_rate'] = user_event_counts['conversion_rate'].round(2)

# NaN or inf 처리 (view가 0인 경우)
user_event_counts['conversion_rate'].replace([float('inf'), float('nan')], 0, inplace=True)

# 저장
user_event_counts.to_csv('user_conversion_rates.csv', index=False)

# 결과 출력
print("✅ 'user_conversion_rates.csv' 파일로 저장 완료!")
print("\n📊 유저별 전환율 상위 10명:")
print(user_event_counts.sort_values(by='conversion_rate', ascending=False).head(10))


C:\Users\HR\AppData\Local\Temp\ipykernel_8160\3372606268.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  user_event_counts['conversion_rate'].replace([float('inf'), float('nan')], 0, inplace=True)


✅ 'user_conversion_rates.csv' 파일로 저장 완료!

📊 유저별 전환율 상위 10명:
event_type                                       user_key  cart  purchase  \
1066920     54351257616e3af43-c113-4a04-afa5-83f4f9f735b8     0       129   
1886136     572631412a133acd7-8cf6-e069-e38d-303c4104c3da    79       123   
1828117     570593977cba9985f-3309-4748-a6c5-93ea72a62393     1        97   
1107621     54658041484b7a4ab-fd4f-432b-b974-60e53ba4a417    47        93   
961164      535670409d7e85cdb-8063-4a11-a5a2-47fa69946695    96        93   
1584366     563485138f2697992-7694-a5c5-728b-a91f4285477b     0        92   
404737      4538210245f37bad7-9cad-4da3-b4a9-03cf798c3975     0        90   
33035       243184747ad223e08-512f-4acc-9a8d-56b3cdabbcf8   107        82   
211101      398149664096daa6e-aab6-49be-94d7-244ba38f7cc1     1        82   
1369698     557790271e77498f5-90fb-4df9-b6c9-b9c28612405f     0        78   

event_type  remove_from_cart  view  conversion_rate  
1066920                   10     1    